## Assignment 1
### Group 14 

> Github repository: https://github.com/LivDreyer/CSS24.git

> History of commits:

> Contribution:

In [40]:
#Importations 
import requests 
from bs4 import BeautifulSoup as BS
import pandas as pd 

## Part 1: Web-scrabing 

In this part, we have included keynote speakers, chairs, authors of parallel talks, authors of posters, the names of researchers from the programme committee of the conference, and the organizers of tutorials. 

#### Names from the program: Plenary Speakers, Keynote Speakers, Authors of Parallel Talks, Authors of Posters, and Chairs
#### Link: https://ic2s2-2023.org/program

In [41]:
#Get html to py

link = "https://ic2s2-2023.org/program"
r = requests.get(link)
soup = BS(r.content)

In [55]:
# Plenary, parallel talks, posters speaker
# Name of final list: ppp_names

#find programs from each day 
plen = soup.find_all("ul", {"class":"nav_list"})

#find all names from the day 
names_plen = []
names_plen_text = []
for i in range(len(plen)): 
    names_plen.append(plen[i].find_all("i"))

for i in range(len(names_plen)):
    for j in range(len(names_plen[i])):
        names_plen_text.append(names_plen[i][j].text)

plen_names_lst = [name for names in names_plen_text for name in names.split(', ')]

ppp_names = []
for i in range(len(plen_names_lst)): 
    ppp_names.append(plen_names_lst[i].title())


for name in ppp_names:
    if '\u201a' in name:
        ppp_names.remove(name)

#print(ppp_names)

In [56]:
# Keynote speakers 
# Name of final list: key_names

table = soup.find_all("table", {"class": "tutorials"})
table = table[0].find_all("td")

table_text = []
for i in range(len(table)):
    txt = table[i].text
    if "Keynote" in txt: 
        table_text.append(txt)

key_names = []
for i in table_text: 
    key_names.append(i[10:])


#print(key_names)

In [57]:
# Chairs
# Name of final list: chair_names


chair = soup.find_all("div", {"class":"col-9 col-12-medium"})

#find chairs
names_chairs = []
names_chairs_text = []
for i in range(len(chair)): 
    names_chairs.append(chair[i].find_all("h2"))

for i in range(len(names_chairs)):
    for j in range(len(names_chairs[i])):
        names_chairs_text.append(names_chairs[i][j].text)


chr = []

for i in names_chairs_text:
    if "Chair" in i: 
        chr.append(i)

chair_names = []
for i in chr: 
    idx = i.find("Chair:")
    chair_names.append(i[idx + len("Chair:"):].strip())
    
    
#print(chair_names)

#### Names from the programme committee 
#### Link: https://ic2s2-2023.org/program_committee

In [58]:
link_pc = "https://ic2s2-2023.org/program_committee"
r_pc = requests.get(link_pc)
soup_pc = BS(r_pc.content)

In [59]:
# Programme Committee 
# Name of final list: pcom_names 

pc = soup_pc.find_all("article", {"class":"box post"})

#find all names from the day 
pc_names = []
pc_text = []
for i in range(len(pc)): 
    pc_names.append(pc[i].find_all("li"))

for i in range(len(pc_names)):
    for j in range(len(pc_names[i])):
        pc_text.append(pc_names[i][j].text)


pcom_names = []
for i in range(len(pc_text)):
    lst = pc_text[i].split(" (")
    pcom_names.append(lst[0])

#print(pcom_names)

#### Names from the Tutorials
#### Link: https://ic2s2-2023.org/tutorials

In [60]:
link_t = "https://ic2s2-2023.org/tutorials"
r_t = requests.get(link_t)
soup_t = BS(r_t.content)

In [61]:
# Tutorials 
# Name of final list: tut_names_f

tut = soup_t.find_all("div", {"class": "col-5 col-12-medium"})

#find all names from the day 
tut_names = []
tut_text = []
for i in range(len(tut)): 
    tut_names.append(tut[i].find_all("li"))

for i in range(len(tut_names)):
    for j in range(len(tut_names[i])):
        tut_text.append(tut_names[i][j].text)

#print(tut_text)

tut_names_f = []
for i in range(len(tut_text)):
    lst = tut_text[i].split(",")
    tut_names_f.append(lst[0])

#print(tut_names_f)

### Collect all names in one final string and process 

In [77]:
# Collect all names in one list of strings 

all_names = chair_names + key_names + ppp_names + pcom_names + tut_names_f
#print(all_names)

#Capitalize every name 
all_names_final = []
for i in range(len(all_names)): 
    all_names_final.append(all_names[i].title())

all_names_final = list(set(all_names_final))


#Pandas DF 
names_DF = pd.DataFrame(all_names_final, columns=['Researcher Name'])
names_DF = names_DF.sort_values(by='Researcher Name')

#Split names into first name and last name, dropping middle names in all formats: "John Name Doe" or "John N. Doe"
names_DF['first_name'] = names_DF['Researcher Name'].str.split().str[0]
names_DF['last_name'] = names_DF['Researcher Name'].str.split().str[-1]

#Remove duplicates of names such as "John Name Doe" or "John N. Doe"
names_DF.drop_duplicates(subset=['first_name', 'last_name'], inplace=True)

#Save first column (names) to file
names_DF['Researcher Name'].to_csv('/Users/livdreyerjohansen/Desktop/CSS/Names_DF_Ass1.csv', index=False)

#Find number of researchers

print("Number of names including optional exercises: ",len(names_DF))

Number of names including optional exercises:  1620


### How many unique researchers did you get? 

As seen above, we got 1620 unique names of researchers when including the optional exercises. 

### Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words).

When initially inspecting the webpage to understand how it was structured, and where we should look, we relied on visual cues signaling what html-code corresponded to the data we wanted. Working with retrieving names of researchers from the program, we structured the retrieval in three parts, as we recognized different patterns for each type of name. We repeated this process for the optional exercise. For each part we used the BeautifulSoup’s .find_all() method to extract the wanted names, and lastly combined every list of names to one final list. For the final list, we capitalized each name, and then used the set() method to remove apparent duplicates. We then created a Pandas Dataframe, where we split every name into first and last name excluding the middle name. It was then possible to remove rows with identical first and last name, letting us create a csv.file with unique “Researcher Names”.

## Part 2: Ready Made vs Custom Made Data

*Exercise: Ready made data vs Custom made data In this exercise, I want to make sure you have understood they key points of my lecture and the reading.*

**What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book (answer in max 150 words).**

Centola’s experiment allows the investigation of diffusion processes with no influence from confounding variables such as advertising or social reinforcement from outside of the experiment. Though generally desired it’s hard to apply the results directly on real-word settings. Despite a smaller sample size, Centola could repeat the experiment to reproduce tendencies of spread of information. A problem of volunteer bias could however possibly arise from how he recruited participants.
In Nicolaides’s study the large sample size and the non-reactivity of users constitute an advantage when investigating contagion as it can mirror real-world conditions. However, as mentioned in “Bit by bit” 2.3.3, non-reactivity does not ensure that the data directly reflects behavior. Users of the fitness-app is not forced to participate even though they have connections on the app, and to paraphrase from the book: “It’s not that I am not exercising, I am just not posting it on thefitness- app”. 


**How do you think these differences can influence the interpretation of the results in each study? (answer in max 150 words)**

Centola’s findings are based on an artificial environment that find significant results mapping the “essence” of behavioral contagion, however, the reactivity of participants could influence the interpretation of the results. Additionally, as mentioned, the participants were recruited from health-sites which could promote the behavior Centola finds considering his experiment is also concerned with health behavior. 

Depending on how Nicolaides performs the linear regression to examine how one user’s exercise might influence their connections’ exercise on the fitness-tracker app, it might not mirror the actual contagion happening. If people are active users who follow their friends, but do not post their exercise themselves, the results of the linear regression may reflect this more than the actual influence users have on their connections.
